In [2]:
%run ../load_files.ipynb

In [3]:
%run ../data_all_matrixes.ipynb

,profile,age,gender,MS_HV,MS_type,EDSS,disease_duration,education,connectivity,degree,...,MFIS,SF36_mental,SF36_physical,DTI_FA,DTI_L1,DTI_MD,DTI_RX,RAW,FUNC,LS
ID,,,,,,,,,,,,,,,,,,,,,
FIS_001,none,45.506,female,MS,RR,1.5,16.772,secondary school,****,35.842105,...,8.0,42.97,54.45,"[[0.0, 0.439328825873252, 0.4625400010374732, ...","[[0.0, 0.001062246374944052, 0.001097384692440...","[[0.0, 0.000705840751585395, 0.000715541904089...","[[0.0, 0.0005276379438376123, 0.00052462053555...","[[0.0, 0.03552844351216702, 0.0006652241379404...",NaN,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
FIS_002,none,42.981,male,MS,RR,1.5,10.872,primary school,****,30.368421,...,4.0,60.25,53.15,"[[0.0, 0.4382497080416883, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001055207424665258, 0.0, 0.0, 0.0, 0....","[[0.0, 0.000703053658342256, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0005269767649653327, 0.0, 0.0, 0.0, 0...","[[0.0, 0.028578488896818185, 0.0, 0.0, 0.0, 0....",NaN,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
FIS_004,none,52.239,female,MS,SP,6.0,21.828,graduate,****,30.736842,...,12.0,56.38,39.40,"[[0.0, 0.3322280520926731, 0.0, 0.0, 0.0, 0.37...","[[0.0, 0.001171813137463012, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0008873735393089818, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0007451537309838455, 0.0, 0.0, 0.0, 0...","[[0.0, 0.06991186050430216, 0.0, 0.0, 0.0, 0.0...",NaN,"[[0.0, 1009.125, 0.0, 0.0, 0.0, 421.875, 0.0, ..."
FIS_005,none,36.672,male,MS,RR,1.0,11.619,post-graduate,****,27.736842,...,0.0,59.19,55.81,"[[0.0, 0.4078810391594139, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001044529380979277, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0007169208696827965, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0005531166160359597, 0.0, 0.0, 0.0, 0...","[[0.0, 0.02578800302151145, 0.0, 0.0, 0.0, 0.0...",NaN,"[[0.0, 20.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
FIS_007,attention,31.233,female,MS,RR,2.0,13.928,post-graduate,****,30.184211,...,5.0,47.17,49.24,"[[0.0, 0.4314795444662643, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001069484424799801, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0007114571565313628, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0005324435254387877, 0.0, 0.0, 0.0, 0...","[[0.0, 0.035325089806942316, 0.0, 0.0, 0.0, 0....",NaN,"[[0.0, 60.75, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."


(98, 39)


In [4]:
%run ../y_multilevel.ipynb

,attention,memory
ID,,
FIS_001,0,0
FIS_002,0,0
FIS_004,0,0
FIS_005,0,0
FIS_007,1,0


(98, 2)

In [5]:
y = y_values.memory

In [6]:
# Training and test subsamples

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.333, random_state=2018, stratify=y)

In [7]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(65, 39)
(33, 39)
(65,)
(33,)


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from neudataload.transformer import CombineMatrixTransformer, SpreadOutMatrixTransformer, FeatureMatrixTransformer
from neudataload.utils import all_combinations

# Pipeline configuration
pipe = Pipeline([
    ('combining', CombineMatrixTransformer(column_name='combined')),
    ('spread_out', SpreadOutMatrixTransformer(columns=['combined'])),
    ('filter', FeatureMatrixTransformer(matrix_columns=['combined'])),

    ('reduce_dim', PCA()),
    ('classify', SVC(kernel='poly',decision_function_shape='ovo'))
])

In [9]:
# Parametrized values
N_FEATURES_OPTIONS = range(1, 500, 10)
PENALTY = ['l1', 'l2']
REGULATION_C = np.linspace(1, 500, 50)
MATRIXES = list(all_combinations(['DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', ]))
EXTRA_1D_COLUMS = [None] # + list(all_combinations(available_columns))                            
DEGREE = range(2, 10)

param_grid = { 
        'combining__columns': MATRIXES,
        'filter__columns': EXTRA_1D_COLUMS,
        
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
        'classify__C' : REGULATION_C,
        'classify__degree': DEGREE,
}


In [10]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Search the best parameter combination with Cross Validation 
cv = StratifiedKFold(n_splits=3, shuffle=True)

grid = RandomizedSearchCV(pipe, cv=cv, n_jobs=-1, param_distributions=param_grid, 
                          verbose=3,
                          scoring='accuracy',
                          n_iter = 150)


In [11]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
[CV] reduce_dim__n_components=91, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=3, classify__C=337.06122448979596 
[CV] reduce_dim__n_components=91, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=3, classify__C=337.06122448979596 
[CV] reduce_dim__n_components=91, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=3, classify__C=337.06122448979596 
[CV] reduce_dim__n_components=171, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=9, classify__C=11.183673469387756 
[CV] reduce_dim__n_components=171, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=9, classify__C=11.183673469387756 
[CV] reduce_dim__n_components=171, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=9, classify__C=11.183673469387756 
[CV] reduce_dim__n_components=281, filte

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.5s


[CV]  reduce_dim__n_components=471, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), classify__degree=4, classify__C=62.102040816326536, score=0.6363636363636364, total=   1.2s
[CV]  reduce_dim__n_components=471, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__degree=2, classify__C=306.51020408163265, score=0.7272727272727273, total=   1.0s
[CV]  reduce_dim__n_components=471, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX'), classify__degree=4, classify__C=62.102040816326536, score=0.6666666666666666, total=   1.1s
[CV] reduce_dim__n_components=181, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__degree=2, classify__C=82.46938775510205 
[CV] reduce_dim__n_components=181, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__degree=2, classify__C=82.46938775510205 
[CV] reduce_dim__n_components=181, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), c

[CV]  reduce_dim__n_components=201, filter__columns=None, combining__columns=('DTI_FA',), classify__degree=5, classify__C=214.8571428571429, score=0.6666666666666666, total=   1.3s
[CV] reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=8, classify__C=255.5918367346939 
[CV] reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=8, classify__C=255.5918367346939 
[CV]  reduce_dim__n_components=71, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), classify__degree=3, classify__C=31.551020408163268, score=0.6363636363636364, total=   1.5s
[CV] reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=8, classify__C=255.5918367346939 
[CV]  reduce_dim__n_components=241, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX'), classify__degree=9, classify__C=184.3061224489796, score=0.6363

[CV]  reduce_dim__n_components=81, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD'), classify__degree=3, classify__C=489.8163265306123, score=0.6818181818181818, total=   2.4s
[CV] reduce_dim__n_components=191, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX'), classify__degree=5, classify__C=1.0 
[CV] reduce_dim__n_components=191, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX'), classify__degree=5, classify__C=1.0 
[CV]  reduce_dim__n_components=301, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__degree=8, classify__C=296.32653061224494, score=0.6363636363636364, total=   2.4s
[CV] reduce_dim__n_components=191, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX'), classify__degree=5, classify__C=1.0 
[CV]  reduce_dim__n_components=31, filter__columns=None, combining__columns=('DTI_MD',), classify__degree=9, classify__C=235.22448979591837, scor

...

[CV] reduce_dim__n_components=381, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=9, classify__C=62.102040816326536 
[CV]  reduce_dim__n_components=451, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=8, classify__C=62.102040816326536, score=0.6363636363636364, total=   0.9s
[CV]  reduce_dim__n_components=451, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=8, classify__C=62.102040816326536, score=0.6363636363636364, total=   1.3s
[CV] reduce_dim__n_components=381, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=9, classify__C=62.102040816326536 
[CV] reduce_dim__n_components=171, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__degree=9, classify__C=153.75510204081633 
[CV]  reduce_dim__n_components=451, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD'), classify__degree=8, classify__C=62.102040816326536, score=0

[CV] reduce_dim__n_components=441, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__degree=6, classify__C=469.44897959183675 
[CV] reduce_dim__n_components=441, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__degree=6, classify__C=469.44897959183675 
[CV]  reduce_dim__n_components=301, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__degree=5, classify__C=316.6938775510204, score=0.6666666666666666, total=   1.3s
[CV] reduce_dim__n_components=241, filter__columns=None, combining__columns=('DTI_FA',), classify__degree=2, classify__C=459.26530612244903 
[CV]  reduce_dim__n_components=301, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__degree=5, classify__C=316.6938775510204, score=0.6363636363636364, total=   1.5s
[CV] reduce_dim__n_components=241, filter__columns=None, combining__columns=('DTI_FA',), classify__degree=2, classify__C=459.26530612244903 
[CV]  reduce_dim__n_components=431, fi

[CV] reduce_dim__n_components=181, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__degree=8, classify__C=174.12244897959187 
[CV]  reduce_dim__n_components=421, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX'), classify__degree=9, classify__C=489.8163265306123, score=0.6363636363636364, total=   1.1s
[CV]  reduce_dim__n_components=421, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX'), classify__degree=9, classify__C=489.8163265306123, score=0.6666666666666666, total=   1.0s
[CV]  reduce_dim__n_components=271, filter__columns=None, combining__columns=('DTI_L1',), classify__degree=7, classify__C=316.6938775510204, score=0.6363636363636364, total=   1.2s
[CV]  reduce_dim__n_components=181, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX'), classify__degree=8, classify__C=174.12244897959187, score=0.6363636363636364, total=   1.4s
[CV]  reduce_dim__n_components=271, fi

[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  1.9min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
          error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('combining', CombineMatrixTransformer(column_name='combined', columns=None,
             op=<function mean at 0x7f9ca805be18>)), ('spread_out', SpreadOutMatrixTransformer(columns=['combined'], symmetric=True)), ('filter', FeatureMatrixTransformer(columns=None, matrix_columns=['combined'])), ...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
          fit_params=None, iid=True, n_iter=150, n_jobs=-1,
          param_distributions={'combining__columns': [('DTI_FA',), ('DTI_L1',), ('DTI_MD',), ('DTI_RX',), ('DTI_FA', 'DTI_L1'), ('DTI_FA', 'DTI_MD'), ('DTI_FA', 'DTI_RX'), ('DTI_L1', 'DTI_MD'), ('DTI_L1', 'DTI_RX'), ('DTI_MD', 'DTI_RX'), ('DTI_FA', 'DTI_L1', 'DTI_MD'), ('DTI_FA', 'DTI_L1', 'DTI_RX'), ('DTI_FA...3, 459.26531, 469.44898, 479.63265,
       489.81633, 500. 

In [12]:
%run ../grid_results.ipynb

reduce_dim__n_components = 471
filter__columns = None
combining__columns = ('DTI_FA', 'DTI_L1', 'DTI_RX')
classify__degree = 2
classify__C = 306.51020408163265
Test
Accuracy in test: 72.73%
             precision    recall  f1-score   support

          0       0.73      0.90      0.81        21
          1       0.71      0.42      0.53        12

avg / total       0.72      0.73      0.71        33

Train
Accuracy in train: 100.00%
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        42
          1       1.00      1.00      1.00        23

avg / total       1.00      1.00      1.00        65

